# Práctica 7.1. Generación de texto con LSTM

Como primera toma de contacto con modelos generativos, en esta práctica vamos a ver cómo generar texto aleatorio con LSTM.


## 1. Implementando generación de texto con LSTM a nivel de caracter

Lo primero que necesitamos son muchos datos de texto que podamos usar para aprender un modelo de lenguaje. Podrías usar cualquier archivo de texto suficientemente grande o un conjunto de archivos de texto como Wikipedia, El Señor de los Anillos, etc. En este ejemplo utilizaremos algunos de los escritos de Nietzsche, el filósofo alemán de finales del siglo XIX (traducido al inglés). El modelo lingüístico que aprenderemos será, por lo tanto, específicamente un modelo del estilo de escritura y de los temas de elección de Nietzsche, en lugar de un modelo más genérico de la lengua inglesa.

In [1]:
import keras
keras.__version__

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

'2.2.4'

## 2. Preparando los datos

Comencemos descargando el corpus y convirtiéndolo a minúsculas:

In [2]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 600893


A continuación, extraeremos secuencias parcialmente superpuestas de longitud `maxlen`, las codificaremos con one-hot encoding y las empaquetaremos en una matriz 3D Numpy `x` con shape `(sequences, maxlen, unique_characters)`. Simultáneamente, preparamos una matriz `y` que contiene los objetivos correspondientes: los caracteres codificados con one-hot que vienen justo después de cada secuencia extraída.

In [3]:
# Longitud de las secuencias de caracteres extraídas
maxlen = 60

# Muestreamos una nueva secuencia cada `step` caracteres
step = 3

# Esto almacena nuestras secuencias extraídas
sentences = []

# Esto almacena los targets (el siguiente caractar)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# Lista de caracteres únicos en el corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# El diccionario que asigna caracteres únicos a su índice en `chars`.
char_indices = dict((char, chars.index(char)) for char in chars)

# A continuación, codificación one-hot de caracteres en arrays binarios
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


## 3. Construyendo la red

Nuestra red es una única capa `LSTM` seguida de un clasificador `Dense` y softmax sobre todos los caracteres posibles. Pero notemos que las redes neuronales recurrentes no son la única manera de generar datos de secuencias; los convnets 1D también han demostrado ser extremadamente exitosos en los últimos tiempos.

In [4]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


Dado que nuestros objetivos están codificados con one-hot, utilizaremos `categorical_crossentropy` como función de pérdida para entrenar al modelo:

In [5]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## 4. Entrenamiento del modelo lingüístico y muestreo del mismo

Dado un modelo entrenado y un fragmento de texto de semilla, generamos nuevo texto repetidamente:

* 1) Tomando del modelo una distribución de probabilidad sobre el carácter siguiente dado el texto disponible hasta el momento.
* 2) Revaluación de la distribución a una determinada "temperatura".
* 3) Muestreo aleatorio del siguiente carácter de acuerdo con la distribución reponderada
* 4) Añadir el nuevo carácter al final del texto disponible

Este es el código que usamos para reponderar la distribución de probabilidad original que sale del modelo, y extraer un índice de caracteres de él (la "función de muestreo"):

In [6]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Finalmente, este es el bucle en el que entrenamos y generamos texto repetidamente. Comenzamos a generar texto usando un rango de temperaturas diferentes después de cada época. Esto nos permite ver cómo evoluciona el texto generado a medida que el modelo comienza a converger, así como el impacto de la temperatura en la estrategia de muestreo.

In [7]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    # Adaptar el modelo para 1 época a los datos de entrenamiento disponibles
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Seleccionar una semilla de texto al azar
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # Generamos 400 caracteres
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
200278/200278 [==============================] - 62s 308us/step - loss: 1.9767
--- Generating with seed: "the beginning, and that even the severest way of life may be"
------ temperature: 0.2
the beginning, and that even the severest way of life may been the exception of the something and the conternonce of the sense of the sumplent of the same and the says and contempt of the soul as a more one which as a this and the exound and the sand the soul of the schereand the schould and the existing the spirits of the conterning the soul and the soul the existing the spirits of the conterning and the sense and the soul and the soul and the sensity of 
------ temperature: 0.5
 and the sense and the soul and the soul and the sensity of the contempt from then thear the exceptions of the experient of the conternanition of the soness of the soul does the who are the sain them in the existente and can one as the
schould in the secrem

d new exeed a german in the consests and ane far one is previously, how all indean or for into wild, three
withny the stated without higher. whether and assyous
continuoy, profounda--adarcy--ourn aver--which, while
best witholy
intellection upon the
ellsued and patilitions, i oncivered--in order ineld as laok ochseless the sees andadwance. have
sothity of modern the qui estimatesm of
dea virtues and "manivorable here" in these rosped to me, oh theirms of
a
------ temperature: 1.2
nd "manivorable here" in these rosped to me, oh theirms of
all averis, why one ha rease." these adart of condogadd, as preisht
acts how for supjing the spicies,, in their facts
and origorow with they has on
mudip, comper one old an agiss, for course, a
supporthd
vassing no ichipse ebli extent of their intoldenow theo: throwhes trantsk
mista-been muss holy concescure of "intiolbje, that must weight: the very supertion, manigiblenels ase, aconred to be power
epoch 9
Epoch 1/1
200278/200278 [=====================

wounds and the sense of the sense and sense of the spirit in the same to the sense of the same to the sense of the sense of the sense and more spirit and something that is the strength and the sense of the sense of the conscience of the same to the sense and more such a souls and the strength and in the sense of the sense of the sense of the sense of the state the sense of the strength and in the sense 
------ temperature: 0.5
nse of the state the sense of the strength and in the sense of a such a word and religion with the fact the should not be distrust of the antiched to the last contrary of disposed the fact that is nothing of the hard to the same is a conteine from the even profoundly from the one of the result and concerned to the steens and some philosophy to the god and more saint than the self-and every strength of the spirit which he as the more and a lower of dece to 
------ temperature: 1.0
h of the spirit which he as the more and a lower of dece to reverence with christian

s had valuation; something, let us the thing under new that reverous of
kan laugh over nowadays with which a would a spide were pleasure, untchwpes of conceiva"
to
lives un
lihefula", nowher one riches,
the there illudeg to gally that cause xiathing, )bjeclible fouthming of the feetmor but commendage-thlishest,
or alone, it means of success's,--but emotion of
undecaman--in huperdoun.
the master vomive perhaps in
radication the medious will
might shoin o as
epoch 24
Epoch 1/1
200278/200278 [==============================] - 61s 304us/step - loss: 1.3262
--- Generating with seed: "to be able
to vent its passion upon them: youth in itself ev"
------ temperature: 0.2
to be able
to vent its passion upon them: youth in itself every the same the greatest and the same disclassion of the same the conscience of the spirit in the same the spirit and also the same the same the same delicate and the conscience and solitude, and also the subject that the same the same the problem of the same the sen

/home/miguel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


ntereux-t? all "willing, and virtues without threg, as to dominaty and misunder the other kill-ty".


14ych. the
eciv
epoch 28
Epoch 1/1
200278/200278 [==============================] - 61s 304us/step - loss: 1.3179
--- Generating with seed: "e term: the strong man will necessarily in individual and
ex"
------ temperature: 0.2
e term: the strong man will necessarily in individual and
exception of the reality of the world it is the same the state of the will to which the sensual the world and soul and something of the same to the same the standard and the sense of the same to the same the conditional sensual of the same to the result of the same the spirit that the same the same the greatest and superiorism that the same to the same the standard and suffering and superiorism and 
------ temperature: 0.5
 to the same the standard and suffering and superiorism and more contempt to the more faith in the will in the same still such an interests of the same origin that the contempt to the sa

be quiency neundess himself, that doyen a, "knowed-espess as "ialat
invavied, and who reafise.

roking--vent dwardingswer
means or demoviuping orares inprotrad of rekism, and bhilder themselves--seldof, every
volucimating same churge.


1op concerning the pposicismfound. unders a philan that likes bood. mier, now natural "be hopses there, at the changedviony of "natura.


12

ekins--"an advaity. notorative, wh repare yeehly, ho
epoch 32
Epoch 1/1
200278/200278 [==============================] - 61s 304us/step - loss: 1.4417
--- Generating with seed: "ty. in
every conceivable small and great experience he belie"
------ temperature: 0.2
ty. in
every conceivable small and great experience he belienon and a great to be addated to the soul have to thinks and the contucathed and belief and sense than a pechte is not to beloones may be the soul will the chrished and belief and seem to be action of the sucwudo-st and brings of the proposician, and a senses of the contuchtswwqul's a: the seems 

one a t n ote tale t  a toetean en aon ontate a  to ton  oe t te ol tononantet t toe teeinte ane  toenaninto te  otete n an  on an t tono a t o en o tele t  ee le oolino  anne ti  tee it an teto ele tin tene an onlote e to  ant annte ti aoe o t teetoonte ia te anen t te elo tentan t et en ten te aen tan l to t te too iene t an one t a aon ie to a oone ateiel tot inte t e aon oe en e tt ten ee t tot one tonti t t ne ato aen tal tel tot an ta aeo  tonta an t
------ temperature: 1.0
t one tonti t t ne ato aen tal tel tot an ta aeo  tonta an t tioe iene aen  oneneleene t  oe iollta anot ent  ee  on oie e antine annealinie ont anlenanoe  an a te e a t toneinnean en o anne oo ontano  netenetini t ainoet enonteni i on t notottlie inalt ee  oel attl  te olee t tt o ae eeoleot  eeat noneano tente  on ianointe teet  n  an tole t oalontenel teatto t tonin in iinottont te ae ataineeetoon ent int  nl  otooanoininellale aneot  teaine anete oo oi
------ temperature: 1.2
oon ent int  nl  otooanoininel

alinit an ar reirnnarnennaoaeri taaneet e annint atti oentiaaitn anenantli ain ooin ato oin o antteeelanaint alatnoonlaaaintno tatolaianiinlitolaettiean na nia rin onie ininitt ei lotieniatte itierelanoeritlitee er  t on nitraar nnae arl atietlnotaaneontiininaonaneitittt ialetnt itanitreta ananer niat  er tintttii iai nleio aniinaoete ananattitlir t ttiarniteianatianen tilaa lat oaineee ono totinortaiatnti l oratliini en t ooal intannaerielir tiian arilnio
epoch 47
Epoch 1/1
200278/200278 [==============================] - 61s 304us/step - loss: 7.1917
--- Generating with seed: "rit
enjoys therein its craftiness and its variety of disguis"
------ temperature: 0.2
rit
enjoys therein its craftiness and its variety of disguis te t t e t anerte tae tene an  titt t e  te entit te oret t tre t re et ti t t t ateeane er me ane eret t ie tr tin tat at er te tane t ane ot te real te  te t re t t et te  a te t at t ter t te tnt t tit te tr tte t te lr tnear te ent t ae itet rere t t te t t= tr t

e aie ao tore ta tit t e port t t tie t ot t t tt t tir to  e ke torett anon t te ant t a t tire te ton anielr  eritit anere t t iatat  oe titi o oni t il to ltie at tno inor it t tit oro  t   o t te alele toal ee t one tl t to  iet t ir in   o iat tore tain ti e in t te to ao  ien tirr t  taint ae t o t tiei t tit er t to il o e to t tiet tit t  tit tt or t  t t te  t tht tte tit e t te t to io titt tin toe toni at to tir  tir et o t aiiiter toneo t tt ti
------ temperature: 1.0
itt tin toe toni at to tir  tir et o t aiiiter toneo t tt tin aol tittt ar t en aite t cort  inenor a no t tne tl ittin ottin ae tinniee toret itto a iertrel ratt earlane i trio itne teta t e ot  orte l atait ti tritteet oner oniin t orier t at tineitr teneate tattio olirti altelitat  atoi n tinia tetite t  n in ntirt nline tatei?lo tre ontlinit t tertite  o iorioitror l l in taot trie  o3 arterer tal t iantreertoo teart tt oreri te ant re anttote te eit o
------ temperature: 1.2
 tal t iantreertoo teart tt or

Como puedes ver, una baja temperatura resulta en un texto extremadamente repetitivo y predecible, pero donde la estructura local es muy realista: en particular, todas las palabras (una palabra es un patrón local de caracteres) son palabras reales en inglés. Con temperaturas más altas, el texto generado se vuelve más interesante, sorprendente, incluso creativo; a veces puede inventar palabras completamente nuevas que suenan algo plausibles (como "eterned" o "troveration"). Con una temperatura alta, la estructura local comienza a descomponerse y la mayoría de las palabras parecen cadenas de caracteres semiautomáticas. Sin duda, aquí 0,5 es la temperatura más interesante para la generación de texto en esta configuración específica. Experimenta siempre con múltiples estrategias de muestreo! Un equilibrio inteligente entre la estructura aprendida y la aleatoriedad es lo que hace que la generación sea interesante.

Ten en cuenta que al entrenar un modelo más grande, más largo, con más datos, puedes lograr muestras generadas que se verán mucho más coherentes y realistas que las nuestras. Pero, por supuesto, no esperes generar ningún texto significativo, salvo por casualidad: todo lo que estamos haciendo es muestrear datos de un modelo estadístico de qué caracteres vienen después de qué caracteres. El lenguaje es un canal de comunicación, y hay una distinción entre lo que son las comunicaciones y la estructura estadística de los mensajes en los que se codifican las comunicaciones. Para evidenciar esta distinción, he aquí un experimento de pensamiento: ¿qué pasaría si el lenguaje humano hiciera un mejor trabajo en la compresión de las comunicaciones, como lo hacen nuestros ordenadores con la mayoría de nuestras comunicaciones digitales? Entonces el lenguaje no sería menos significativo, pero carecería de una estructura estadística intrínseca, lo que haría imposible aprender un modelo de lenguaje como el que acabamos de hacer.


## 5. Conclusiones

* Podemos generar datos de secuencia discreta entrenando un modelo para predecir los siguientes tokens dados los tokens anteriores.
* En el caso del texto, este modelo se denomina "modelo lingüístico" y puede basarse en palabras o caracteres.
* El muestreo del siguiente token requiere un equilibrio entre la adhesión a lo que el modelo juzga probable y la introducción de la aleatoriedad.
* Una manera de manejar esto es la noción de _softmax temperature_. Experimenta siempre con diferentes temperaturas para encontrar la más adecuada.